# Defalut Credit Card 💳<br>Inuk
---

In [ ]:
!pip install scikeras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from scikeras.wrappers import KerasClassifier

from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input

## [1] Load Data
---


In [ ]:
colab_path = '/content/drive/MyDrive/Colab Notebooks'
file_path = colab_path + '/python-mldlnl/natural-language-processing/Data/default of credit card clients.csv'

data = pd.read_csv(file_path, header=1).drop(columns="ID")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype
---  ------                      --------------  -----
 0   LIMIT_BAL                   30000 non-null  int64
 1   SEX                         30000 non-null  int64
 2   EDUCATION                   30000 non-null  int64
 3   MARRIAGE                    30000 non-null  int64
 4   AGE                         30000 non-null  int64
 5   PAY_0                       30000 non-null  int64
 6   PAY_2                       30000 non-null  int64
 7   PAY_3                       30000 non-null  int64
 8   PAY_4                       30000 non-null  int64
 9   PAY_5                       30000 non-null  int64
 10  PAY_6                       30000 non-null  int64
 11  BILL_AMT1                   30000 non-null  int64
 12  BILL_AMT2                   30000 non-null  int64
 13  BILL_AMT3                   30000 non-null  int64
 14  BILL_A

In [ ]:
X = data.drop(columns="default payment next month")
y = data['default payment next month']

In [ ]:
category = ["SEX", "EDUCATION", "MARRIAGE"]

for i in category:
    X[i] = X[i].astype("object")

## [2] Preprocessing
---

In [ ]:
def preprocessing(df:pd.DataFrame):
    columns = df.columns

    for col in columns:
        if df[col].dtype == "object":
            ohe = OneHotEncoder(sparse=False)
            ohe_arr = ohe.fit_transform(df[col].values.reshape(-1, 1))
            ohe_df = pd.DataFrame(ohe_arr)

            df = pd.concat([df, ohe_df], axis=1)
            df = df.drop(columns=col)
        else:
            pass

    df = MinMaxScaler().fit_transform(df)

    return df

In [ ]:
X_pre = preprocessing(X)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


## [3] PCA
---

In [ ]:
pca = PCA(n_components=0.95).fit(X_pre)

X_pca = pca.transform(X_pre)

In [ ]:
# 주성분 행렬
print("주성분 행렬 : ", pca.singular_values_)

# 주성분에 투영 후 분산 정도 설명
print("분산 정도 : ", pca.explained_variance_)

# 주성분에 투영 후 분산 정도 설명
print("분산 정도 비율 : ", pca.explained_variance_ratio_)

주성분 행렬 :  [125.9595317  119.21788538 107.88352473  77.37351817  41.88450817
  26.53589807  23.47956079]
분산 정도 :  [0.52887775 0.47377927 0.38797476 0.19956203 0.05847902 0.02347258
 0.01837694]
분산 정도 비율 :  [0.29966878 0.26844929 0.21983138 0.11307436 0.03313495 0.01329986
 0.0104126 ]


## [4] train test split
---

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_pca, y, stratify=y, random_state=42
)

## [5] Grid Search CV
---

In [ ]:
def build_model(n_hidden, n_neurons, n_drop, learning_rate, input_shape=(4,)):
    
    model = Sequential()
    
    model.add(Input(shape=input_shape))
    
    for layer in range(n_hidden):
        model.add(Dense(n_neurons, activation="relu"))
        model.add(Dropout(n_drop))

    model.add(Dense(1, activation='sigmoid'))
    
    return model

In [ ]:
input_shape = (X_train.shape[1],)

clf = KerasClassifier(
    build_model,
    n_hidden=3,
    n_neurons=50,
    n_drop=0.3,
    learning_rate=0.0001,
    input_shape=input_shape,
    loss="binary_crossentropy",
    epochs=5
)

clf.fit(X_pca, y)

Epoch 1/5
938/938 [==============================] - 3s 3ms/step - loss: 0.5068
Epoch 2/5
938/938 [==============================] - 3s 3ms/step - loss: 0.4816
Epoch 3/5
938/938 [==============================] - 3s 3ms/step - loss: 0.4743
Epoch 4/5
938/938 [==============================] - 3s 3ms/step - loss: 0.4716
Epoch 5/5
938/938 [==============================] - 3s 3ms/step - loss: 0.4714


KerasClassifier(
	model=<function build_model at 0x7f2e386a60e0>
	build_fn=None
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=binary_crossentropy
	metrics=None
	batch_size=None
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=5
	n_hidden=3
	n_neurons=50
	n_drop=0.3
	learning_rate=0.0001
	input_shape=(7,)
	class_weight=None
)

In [ ]:
# params = {
#     "optimizer": ["adam", "sgd"],
#     "learning_rate": [0.0001, 0.001, 0.1],
#     "n_hidden": [3,4,5],
#     "n_neurons": [50, 100, 150],
#     "n_drop": [0.3, 0.5, 0.7],   
# }

# gs = GridSearchCV(
#     clf, params, refit=False, cv=3, scoring="accuracy", verbose=True
# )

# gs.fit(X, y)

In [ ]:
# gs.best_params_
best = {'learning_rate': 0.0001,
 'n_drop': 0.3,
 'n_hidden': 3,
 'n_neurons': 50,
 'optimizer': 'adam'}

In [ ]:
clf.score(X_test, y_test)

235/235 [==============================] - 0s 1ms/step


0.8058666666666666